## Import

In [3]:
import pandas as pd
import random
import os
import numpy as np

In [73]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [108]:
from sklearn import metrics 
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

## Data Load

In [5]:
group_dict = {
    "Group_1" : ["X_01", "X_02", "X_05", "X_06"],
    "Group_2" : ["X_03", "X_10", "X_11"],
    "Group_3" : ["X_04", "X_23", "X_47", "X_48"],
    "Group_4" : ["X_07", "X_08", "X_09"],
    "Group_5" : ["X_12"],
    "Group_6" : ["X_13"],
    "Group_7" : ["X_14", "X_15", "X_16", "X_17", "X_18"],
    "Group_8" : ["X_19", "X_20", "X_21", "X_22"],
    "Group_9" : ["X_24", "X_25", "X_26", "X_27", "X_28", "X_29"],
    "Group_10" : ["X_30", "X_31", "X_32", "X_33"],
    "Group_11" : ["X_34", "X_35", "X_36", "X_37"],
    "Group_12" : ["X_38", "X_39", "X_40"],
    "Group_13" : ["X_41", "X_42", "X_43", "X_44"],
    "Group_14" : ["X_45"],
    "Group_15" : ["X_46"],
    "Group_16" : ["X_49"],
    "Group_17" : ["X_50", "X_51", "X_52", "X_53", "X_54", "X_55", "X_56"],
}

In [124]:
# 참고 https://hong-yp-ml-records.tistory.com/80
train_df = pd.read_csv('./train.csv')
# 파생변수 생성전 의미 없는 검사 결과 빼기
train_df = train_df.drop(columns=["X_04", "X_23", "X_47", "X_48","ID"])
# # 파생변수 생성
for i in range(1,18):
    if i != 3 and len(group_dict['Group_%d'%i]) >1 :
        Group_train = train_df[group_dict['Group_%d'%i]]
        train_df['Group_%d mean'%i] = Group_train.mean(axis = 1)
        train_df['Group_%d var'%i] = Group_train.var(axis = 1)
        train_df['Group_%d std'%i] = Group_train.std(axis = 1)
        train_df['Group_%d sum'%i] = Group_train.sum(axis = 1)
        train_df['Group_%d median'%i] = Group_train.median(axis = 1)
train_df = train_df.dropna(axis = 1)


In [125]:
# # 데이터 나누기
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_df, train_size=0.8)

In [126]:
train_x = train_df.filter(regex='[XG]') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = valid_df.filter(regex='[XG]') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature